Use xmltodict to transfer XML into a dictionary, then read into csv file

In [2]:
import xml.etree.ElementTree as ET
import xmltodict
import csv
import re
from six.moves.urllib.parse import urlencode, urljoin
from six.moves.html_parser import HTMLParser
import pandas as pd
import requests
import json
import collections

In [3]:
# from a list of NCTID call the XML files from clinicalTrials.gov 
#get the list of NCTID
def NCTID_input(xlsxfile):
    """
    Get the input NCTID as a list
    :param xlsxfile
    :return the list
    """
    NCTID_list = []
    filepath = "input\\"+xlsxfile
    df = pd.read_excel(filepath, sheet_name='Sheet1')
    NCTID_list = df['NCT Number'].dropna().tolist()
    return NCTID_list


#call the xml from clinicalTrials.gov using request.get()
BASE_URL = "https://clinicaltrials.gov/ct2/show/"

def get_study(nct_id):
    """
    Pull the XML for the study
    :param nct_id:
    :return:
    """
    nct_id=nct_id.strip()
    t = urljoin(BASE_URL, nct_id)
    full_url = t + "?" + urlencode(dict(displayxml=True))
    response = requests.get(full_url)
    if not response.status_code == 200:
        raise ValueError("Unable to load study {}".format(nct_id))
    return response.text

# primary outcome info 
#  -- if more than 1, the primary outcome is a list of dictionaries 
#  -- if is 1, the primary outcome is a dictionary --> not yet tested
def outcome(tag):
    """
    Pull the outcomes information from XML file
    :param tag_name
    :return 1.counts of the tags, 2. subtag content (measure, timeframe,description) as a list 
    """
    outcome_list =[]
    if type(content[tag]) is list:
        for i in content[tag]:
            #i = {key: value.replace('\n','') for key, value in i.items()}
            outcome_content = json.dumps(i) 
            outcome_list.append(outcome_content)
        outcome_count = len(content[tag])
    else:
        for j in content[tag]:
            outcome_list.append(j+":"+content[tag][j])
            #outcome_list.append(j+":"+content[tag][j].replace('\n',''))
        outcome_count = 1
    outcome_value = '|'.join(outcome_list)
    return(outcome_count,outcome_value)

In [15]:
NCTIDfile = input('please give the file name contains NCTID list:' )
outfilename = input('please give the file name for output: ')

outputfile = "output\\"+outfilename

NCTID_list = NCTID_input(NCTIDfile)
print (str(len(NCTID_list))+' files need to be processed.')

content_dict = {}

k = 0
for i in NCTID_list:
    xml_str= get_study(i)
    #the get_study function returns xml in string. need to read the tree from a string.
    tree = ET.ElementTree(ET.fromstring(xml_str))
    root = tree.getroot()
    xmlstr = ET.tostring(root, encoding='utf8', method='xml')
    xmlstr = xmlstr.replace(b'<?xml version=\'1.0\' encoding=\'utf8\'?>\n', b'') 
    xmlstr = xmlstr.replace(b'\n',b'')
    data_dict = xmltodict.parse(xmlstr)
    
    #only one key in data_dict --> clinical study
    for key in data_dict:
        content=data_dict[key]
    
    # getting nct_id
    content_dict['NCTID'] = content['id_info']['nct_id']
    #title
    content_dict['Title'] = content['brief_title']
    # acronym
    if 'acronym' in content:
        content_dict['Acronym'] = content['acronym']
    else:
        content_dict['Acronym'] = "None"
    # status
    content_dict['Status'] = content['overall_status']
    #oversight_info
    if 'oversight_info' in content:
        if 'has_dmc' in content['oversight_info']:
            content_dict['dmc_info'] = content['oversight_info']['has_dmc']
        else:
            content_dict['dmc_info'] = "None"
    else:
        content_dict['dmc_info'] = "None"
    # results: if tag 'clinical_results' available, then has_results
    if 'clinical_results' in content:
        content_dict['Study Results'] = "has results"
    else:
        content_dict['Study Results'] = "no results"
    # condition - multiple
    content_dict['Condition'] = content['condition']
    # intervention - multiple dictionary - as list
    intervention_list = []
    if type(content['intervention']) is list:
        for j in content['intervention']:
            intervention = j['intervention_type']+":"+j['intervention_name']
            intervention_list.append(intervention)
        content_dict['Intervention'] = '|'.join(intervention_list)
    else:
        content_dict['Intervention'] = content['intervention']['intervention_type']+":"+content['intervention']['intervention_name']
    #lead_sponsor
    content_dict['Sponsor'] = content['sponsors']['lead_sponsor']['agency']
    #lead_sponsor_type maybe equal to "Funded Bys"
    content_dict['Funded Bys'] = content['sponsors']['lead_sponsor']['agency_class']
    #gender under eligibility one tag
    content_dict['Gender'] = content['eligibility']['gender']
    #age
    if content['eligibility']['maximum_age'] != "N/A": 
        max_age = content['eligibility']['maximum_age'] 
    else:
        max_age =""
    content_dict['Age'] = content['eligibility']['minimum_age']+"-"+max_age
    #phase
    if 'phase' in content: 
        content_dict['Phase'] = content['phase']
    else:
        content_dict['Phase'] = "None"
    #study type
    content_dict['Study Type'] = content['study_type']
    #enrollment number
    if type(content['enrollment']) is collections.OrderedDict:
        content_dict['Enrollment'] = content['enrollment']['#text']
    else:
        content_dict['Enrollment'] = content['enrollment']
    #location
    if 'location' in content: 
        if type(content['location']) is list:
            content_dict['Sites number'] = len(content['location'])
        else:
            content_dict['Sites number'] = 1
    else:
        content_dict['Sites number'] = "None"
    #study_design_info
    study_design_info = []
    for j in content['study_design_info']:
        study_design = j+":"+content['study_design_info'][j]
        study_design_info.append(study_design)
    content_dict['Study Design Info'] = '|'.join(study_design_info)
    #primary outcome
    primary_outcome = outcome('primary_outcome')
    content_dict['Primary Outcome Count'] = primary_outcome[0]
    content_dict['Primary Outcome'] = primary_outcome[1]
    #secondary outcome
    if 'secondary_outcome' in content:
        secondary_outcome = outcome('secondary_outcome')
        content_dict['Secondary Outcome Count'] = secondary_outcome[0]
        content_dict['Secondary Outcome'] = secondary_outcome[1]
    else:
        content_dict['Secondary Outcome Count'] = 0
        content_dict['Secondary Outcome'] = "None"
    #arm
    if 'arm_group' in content:
        arm_group = outcome('arm_group')
        content_dict['arm group count'] = arm_group[0]
        content_dict['arm group list'] = arm_group[1]
    else:
        content_dict['arm group count'] = 0
        content_dict['arm group list'] = "None"
    #inclusion_exclusion_criteria
    if 'criteria' in content['eligibility']:
        inclusion_exclusion = content['eligibility']['criteria']['textblock']
        inclusion_exclusion = str(inclusion_exclusion)
        inclusion_exclusion = inclusion_exclusion.replace('\n','')
        inclusion_exclusion = re.sub(' +', ' ', inclusion_exclusion) 
        content_dict['inclusion_exclusion'] = inclusion_exclusion
    else:
        content_dict['inclusion_exclusion'] = "None"
    #mesh term
    if 'intervention_browse' in content:
        if 'mesh_term' in content['intervention_browse']:
            content_dict['mesh_term'] = content['intervention_browse']['mesh_term']
    else:
        content_dict['mesh_term'] = "None"
    #keyword
    if 'keyword' in content:
        content_dict['keyword'] = content['keyword']
    else:
        content_dict['keyword'] = "None"
    print (i+ ' processed!')
    # input the dictionary into a csv file
    with open(outputfile, 'a', encoding = 'utf8') as f:
        for key in content_dict.keys():
            f.write("%s:%s||"%(key,content_dict[key]))
        f.write("\n")
    f.close()
    #dump the dictionary into json file
    jfilename = "output\\json\\"+i+"_data.json"
    with open(jfilename, 'w') as fj:
        json.dump(content_dict, fj,indent=4)
    fj.close()
    k=k+1

print(str(k)+ ' files processed and wrote into the outputfile.')


please give the file name contains NCTID list:CT69.xlsx
please give the file name for output: CT69_xmltodict_out_010303.csv
69 files need to be processed.
NCT02599389 processed!
NCT02710656 processed!
NCT01221610 processed!
NCT01867736 processed!
NCT00696956 processed!
NCT00930813 processed!
NCT01594684 processed!
NCT02498080 processed!
NCT01558505 processed!
NCT03175744 processed!
NCT02963649 processed!
NCT01816412 processed!
NCT02063672 processed!
NCT01566461 processed!
NCT01947478 processed!
NCT01412541 processed!
NCT00987324 processed!
NCT01495533 processed!
NCT01239953 processed!
NCT01239940 processed!
NCT01255956 processed!
NCT00485030 processed!
NCT01093300 processed!
NCT02924857 processed!
NCT02561299 processed!
NCT01858428 processed!
NCT00472472 processed!
NCT02965677 processed!
NCT01247402 processed!
NCT03884257 processed!
NCT02129634 processed!
NCT02772224 processed!
NCT01556542 processed!
NCT02033135 processed!
NCT02812966 processed!
NCT00776906 processed!
NCT02923193 proce